In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/level2-object-detection-level2-cv-16/UniverseNet/configs/universenet/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_trash_002.py')

root='../../dataset/'

epoch = 'epoch_20'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 16

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_trash_002'

# cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=4,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/level2-object-detection-level2-cv-16/UniverseNet/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: ./work_dirs/universenet101_2008d_fp16_4x4_mstrain_480_960_20etrash_002/epoch_20.pth


2022-04-07 05:02:16,516 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.0 is upgraded to version 2.
2022-04-07 05:02:16,518 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.1 is upgraded to version 2.
2022-04-07 05:02:16,520 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.2 is upgraded to version 2.
2022-04-07 05:02:16,525 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.0 is upgraded to version 2.
2022-04-07 05:02:16,526 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.1 is upgraded to version 2.
2022-04-07 05:02:16,527 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.2 is upgraded to version 2.
2022-04-07 05:02:16,537 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.0 is upgraded to version 2.
2022-04-07 05:02:16,538 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.1 is upgraded to version 2.
2022-04-07 05:02:16,539 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.2 is upgraded to version 2.
2022-04-07 05:02:16

In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 24.0 task/s, elapsed: 203s, ETA:     0s

In [6]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.37898162 225.93513 680.3362 317.29428 743....,test/0000.jpg
1,0 0.38596642 131.43404 0.0 503.0035 285.0421 0...,test/0001.jpg
2,0 0.72121966 84.64732 268.99 419.4911 611.7628...,test/0002.jpg
3,0 0.21999255 29.454163 23.364899 1020.54443 10...,test/0003.jpg
4,0 0.5132388 410.29565 409.31332 666.23267 579....,test/0004.jpg
